In [1]:
import os,cv2,random,numpy as np

In [2]:
# Function to load images from a directory
def load_images_from_directory(directory):
    data = []
    for root, _, files in os.walk(directory):
        for filename in files:
            if filename.endswith(".jpg") or filename.endswith(".png"):
                img = cv2.imread(os.path.join(root, filename), cv2.IMREAD_GRAYSCALE)
                if img is not None:
                    data.append({'filename': filename, 'image': img})
    return data


In [3]:
import os
import cv2
import random

def apply_rotations_and_save(input_dir="data/preprocess", output_dir="data/rotate", rotation_angles=[45, 90, 135]):
    # Traverse each image in the input directory
    for root, _, files in os.walk(input_dir):
        for filename in files:
            if filename.endswith(('.png', '.jpg', '.jpeg')):
                # Read the image
                img_path = os.path.join(root, filename)
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

                # Apply rotations and save
                for angle in rotation_angles:
                    # Get the image center and dimensions
                    h, w = img.shape
                    center = (w // 2, h // 2)

                    # Calculate the rotation matrix
                    rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)

                    # Compute the new bounding box dimensions to prevent cropping
                    abs_cos = abs(rotation_matrix[0, 0])
                    abs_sin = abs(rotation_matrix[0, 1])
                    new_w = int(2 * h * abs_sin + w * abs_cos)
                    new_h = int(2 * h * abs_cos + w * abs_sin)

                    # Adjust the rotation matrix to account for the translation
                    rotation_matrix[0, 2] += (new_w / 2) - center[0]
                    rotation_matrix[1, 2] += (new_h / 2) - center[1]

                    # Perform the rotation with the new bounding box size
                    rotated_img = cv2.warpAffine(img, rotation_matrix, (new_w, new_h), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=255)

                    # Resize the rotated image back to the original dimensions
                    resized_img = cv2.resize(rotated_img, (w, h))

                    # Save the image in an angle-based folder
                    angle_dir = os.path.join(output_dir, f"{angle}")
                    os.makedirs(angle_dir, exist_ok=True)

                    output_path = os.path.join(angle_dir, filename)
                    cv2.imwrite(output_path, resized_img)

    print(f"Images saved to {output_dir}")

In [5]:
def add_gaussian_noise(image, mean=0, std=10):
    noise = np.random.normal(mean, std, image.shape).astype(np.float32)
    noisy_image = image.astype(np.float32) + noise

    # Clip the pixel values to be in the valid range [0, 255]
    noisy_image = np.clip(noisy_image, 0, 255).astype(np.uint8)

    return noisy_image

In [6]:
import os
import cv2
import numpy as np

def apply_noise_and_save(input_dir="data/preprocess", output_dir="data/noise", noise_levels=[10, 50]):
    # Traverse each image in the input directory
    for root, _, files in os.walk(input_dir):
        for filename in files:
            if filename.endswith(('.png', '.jpg', '.jpeg')):
                # Read the image
                img_path = os.path.join(root, filename)
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

                # Apply noise and save
                for noise_level in noise_levels:
                    # Generate Gaussian noise
                    noise = np.random.normal(0, noise_level, img.shape).astype(np.uint8)

                    # Add the noise to the image
                    noisy_img = cv2.add(img, noise)

                    # Save the noisy image in a noise-level-based folder
                    noise_dir = os.path.join(output_dir, f"noise_{noise_level}")
                    os.makedirs(noise_dir, exist_ok=True)

                    output_path = os.path.join(noise_dir, filename)
                    cv2.imwrite(output_path, noisy_img)

    print(f"Noisy images saved to {output_dir}")

In [8]:
def scale_image(image, scale):
    """
    Scale an image to a specified factor.

    Parameters:
        image (numpy.ndarray): The input image.
        scale (float): Scaling factor (e.g., 0.5 for half size, 1.5 for 1.5x size).

    Returns:
        numpy.ndarray: Scaled image.
    """
    h, w = image.shape[:2]  # Original height and width
    new_w = int(w * scale)
    new_h = int(h * scale)

    # Resize the image to the new dimensions
    scaled_image = cv2.resize(image, (new_w, new_h), interpolation=cv2.INTER_LINEAR)
    return scaled_image

In [9]:
def apply_scaling_and_save(images_data, output_dir="output_scaling", num_per_user=5, scaling_factors=[0.5, 0.75, 1.25, 1.5]):
    user_data = {}

    # Group images by user
    for img_data in images_data:
        user_id = img_data['user_id']
        if user_id not in user_data:
            user_data[user_id] = []
        user_data[user_id].append(img_data)

    # Apply scaling and save
    for user_id, images in user_data.items():
        # Select `num_per_user` images for each user
        selected_images = random.sample(images, min(len(images), num_per_user))

        for img_data in selAected_images:
            img = img_data['image']
            filename = img_data['filename']

            # Randomly select a scaling factor from the provided list
            scale = random.choice(scaling_factors)

            # Apply scaling
            scaled_img = scale_image(img, scale)

            # Save the scaled image in a scale-based folder
            scale_dir = os.path.join(output_dir, f"scale_{str(scale).replace('.', '_')}")
            os.makedirs(scale_dir, exist_ok=True)

            output_path = os.path.join(scale_dir, filename)
            cv2.imwrite(output_path, scaled_img)

    print(f"Scaled images saved to {output_dir}")

In [10]:
images_data = load_images_from_directory("data/isolated_words_per_user", target_size=(256, 128))
apply_scaling_and_save(images_data, output_dir="preprocessing/scaled_images", num_per_user=5, scaling_factors=[0.5, 0.75, 1.25, 1.5])

Scaled images saved to preprocessing/scaled_images
